In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import torch as pt
import time
import sys

sys.path.insert(0, '..')

from NNsolver import NNSolver
from NNarchitectures import DenseNet_g
from problems import DoubleWell
from utilities import get_X_process, plot_NN_evaluation, compute_PDE_loss

%load_ext autoreload
%autoreload 2

device = pt.device('cuda')

In [ ]:
T = 0.5
problem = DoubleWell(d=50, d_1=50, d_2=0, T=0.5, eta=.05, kappa=.1)
problem.compute_reference_solution()
problem.compute_reference_solution_2()
K = 2000
K_batch = 2000
print_every = 500
delta_t = 0.01
sq_delta_t = pt.sqrt(pt.tensor(delta_t))
N = int(np.ceil(T / delta_t))
gradient_steps = (N - 1) * [2000] + [25000]
learning_rates = [0.0002] * (N - 1) + [0.0005]

model = NNSolver(problem, 'DoubleWell', learning_rates=learning_rates, gradient_steps=gradient_steps, NN_class=DenseNet_g, K=K, 
                 K_batch=K_batch, delta_t=delta_t, print_every=print_every, method='implicit')

model.Y_n = [DenseNet_g(problem.d, 1, lr=learning_rates[n], problem=problem).to(device) for n in range(N)] + [problem.g]

In [ ]:
model.train()

In [ ]:
fig = plot_NN_evaluation(model, n=N-2, reference_solution=False, Y_0_true=problem.v_true(problem.X_0[np.newaxis, :], 0))

In [ ]:
pde_loss = compute_PDE_loss(problem, delta_t=0.01, K=1000, Y_n=model.Y_n, vfun=None, testOde=None, seed=44, print_every=10)

In [ ]:
np.mean(pde_loss[:-1])

### compute reference loss

In [ ]:
K = 1000

problem.modus = 'np'
X, xi = get_X_process(problem, K, delta_t, seed=42, t=0, x=problem.X_0)

Y_ref = np.zeros([N + 1, K])
Y_pred = np.zeros([N + 1, K])


for n in range(N + 1):
    for k in range(K):
        Y_ref[n, k] = problem.v_true(X[n, k, :][np.newaxis, :], n * delta_t)

    problem.modus = 'pt'   
    Y_pred[n, :] = model.Y_n[n](pt.tensor(X[n, :, :]).float().to(device)).cpu().detach().squeeze().numpy()
    problem.modus = 'np'   

In [ ]:
np.mean(np.abs((Y_ref - Y_pred) / Y_ref))